In [49]:
import numpy as np
import cvxpy as cp
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('COL774_A5_SVM/train_nls.csv')
X = data.drop('target', axis=1).values
Y = data['target'].values
Y = np.where(Y==0,-1,1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
n, d = X.shape

In [50]:
w = cp.Variable(d)        
b = cp.Variable()        
slack = cp.Variable(n)   

C = 1  
objective = cp.Minimize(cp.norm1(w)/2 + C * cp.sum(slack))

constraints = []
for i in range(n):
    constraints.append(Y[i] * (X[i] @ w + b) >= 1 - slack[i]) 
    constraints.append(slack[i] >= 0) 

problem = cp.Problem(objective, constraints)
problem.solve()


output = {
    "weights": w.value.tolist(),  
    "bias": b.value.item()  
}

with open("svm_weights.json", "w") as json_file:
    json.dump(output, json_file, indent=4)

In [51]:
separable = 0
support_vectors= []
if max(slack.value)<1:
    separable = 1
    for i in range(n):
        margin_distance = Y[i] * (X[i] @ w.value + b.value)
        if np.isclose(margin_distance, 1, atol=1e-4):  
            support_vectors.append(i)

output = {
    "seperable": separable,  
    "support_vectors": support_vectors
}

with open("svm_sv.json", "w") as json_file:
    json.dump(output, json_file, indent=4)